# Thông Tin
1. MSSV: 20520079
2. Họ và tên: Nguyễn Tư Thành Nhân
3. Bài tập: Assignment 4

# Import các thư viện cần thiết

In [1]:
import gym
import numpy as np
import time
from IPython import display

# Một số hàm dùng để chơi game 

In [2]:
def play(env, policy, render=False):
    state = env.reset()
    total_reward = 0
    steps = 0
    done = False
    while not done:
        action = policy[state]
        next_state, reward, done, info = env.step(action)
        total_reward += reward
        steps += 1
        if render:
            env.render()
            time.sleep(0.2)
            if not done:
                display.clear_output(wait=True)
        state = next_state

    return (total_reward, steps)

In [3]:
def play_multiple_times(env, policy, max_episodes):
    success = 0
    list_of_steps = []
    for i in range(max_episodes):
        total_reward, steps = play(env, policy)

        if total_reward > 0:
            success += 1
            list_of_steps.append(steps)

    print(f'Number of successes: {success}/{max_episodes}')
    print(f'Average number of steps: {np.mean(list_of_steps)}')

In [4]:
def policy_evaluation(env, policy, max_iters=500, gamma=0.9):
    # Initialize the values of all states to be 0
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # Update the value of each state
        for state in range(env.observation_space.n):
            action = policy[state]

            # Compute the q-value of the action
            q_value = 0
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * prev_v_values[next_state])

            v_values[state] = q_value # update v-value
        
        # Check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

# Cài đặt thuật toán Value Iteration

In [5]:
def value_iteration(env, max_iters=500, gamma=0.9):
    # initialize
    v_values = np.zeros(env.observation_space.n)

    for i in range(max_iters):
        prev_v_values = np.copy(v_values)

        # update the v-value for each state
        for state in range(env.observation_space.n):
            q_values = []
            
            # compute the q-value for each action that we can perform at the state
            for action in range(env.action_space.n):
                q_value = 0
                # loop through each possible outcome
                for prob, next_state, reward, done in env.P[state][action]:
                    q_value += prob * (reward + gamma * prev_v_values[next_state])
                
                q_values.append(q_value)
            
            # select the max q-values
            best_action = np.argmax(q_values)
            v_values[state] = q_values[best_action]
        
        # check convergence
        if np.all(np.isclose(v_values, prev_v_values)):
            print(f'Converged at {i}-th iteration.')
            break
    
    return v_values

In [6]:
def policy_extraction(env, v_values, gamma=0.9):
    # initialize
    policy = np.zeros(env.observation_space.n, dtype=np.int)

    # loop through each state in the environment
    for state in range(env.observation_space.n):
        q_values = []
        # loop through each action
        for action in range(env.action_space.n):
            q_value = 0
            # loop each possible outcome
            for prob, next_state, reward, done in env.P[state][action]:
                q_value += prob * (reward + gamma * v_values[next_state])
            
            q_values.append(q_value)
        
        # select the best action
        best_action = np.argmax(q_values)
        policy[state] = best_action
    
    return policy

# Cài đặt thuật toán Policy Iteration

In [7]:
def sum_sr(env, V, s, a, gamma):
    """Calc state-action value for state 's' and action 'a'"""
    tmp = 0  # state value for state s
    for p, s_, r, _ in env.P[s][a]:
        tmp += p * (r + gamma * V[s_])
    return tmp

In [8]:
def policy_iteration(env, max_iter = 500, gamma=0.9, theta=1e-8):
    # initialization
    v_values = np.zeros(env.observation_space.n)
    pi = np.zeros(env.observation_space.n, dtype=int)
    # policy Evaluation
    iter = 0
    while iter<max_iter:
      iter+=1
      while True:
        delta = 0
        for s in range(env.observation_space.n):
          v = v_values[s]
          v_values[s] = sum_sr(env,V=v_values, s=s, a=pi[s], gamma=gamma)
          delta = max(delta, abs(v-v_values[s]))
        if delta < theta: break
    # policy Improvement
      policy_stable = True
      for s in range(env.observation_space.n):
        pre_action = pi[s]
        pi[s] = np.argmax([sum_sr(env, V=v_values, s=s, a=a, gamma=gamma) for a in range(env.action_space.n)])
        if pre_action != pi[s]: policy_stable = False
      if policy_stable: 
        print(f'Converged at {iter}-th iteration.')
        break
    return v_values, pi

# Thực nghiệm trên FrozenLake-v0

In [9]:
env = gym.make('FrozenLake-v0')
print(env.P[0][3])
print(env.observation_space.n, env.action_space.n)

[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]
16 4


In [10]:
# Value Iteration
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values)
play_multiple_times(env, optimal_policy, 1000)

Converged at 79-th iteration.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


Number of successes: 713/1000
Average number of steps: 37.523141654978964


In [11]:
# Policy Iteration
optimal_p_values = policy_iteration(env)[1]
play_multiple_times(env, optimal_p_values, 1000)

Converged at 6-th iteration.
Number of successes: 749/1000
Average number of steps: 38.23898531375167


# Thực nghiệm trên FrozenLake8x8-v0

In [12]:
env = gym.make('FrozenLake8x8-v0')
print(env.P[0][3])
print(env.observation_space.n, env.action_space.n)

[(0.3333333333333333, 1, 0.0, False), (0.3333333333333333, 0, 0.0, False), (0.3333333333333333, 0, 0.0, False)]
64 4


In [13]:
# Value Iteration
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values)
play_multiple_times(env, optimal_policy, 1000)

Converged at 117-th iteration.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


Number of successes: 754/1000
Average number of steps: 72.35809018567639


In [14]:
# Policy Iteration
optimal_p_values = policy_iteration(env)[1]
play_multiple_times(env, optimal_p_values, 1000)

Converged at 10-th iteration.
Number of successes: 751/1000
Average number of steps: 71.72170439414114


# Thực nghiệm trên Taxi-v3

In [15]:
env = gym.make('Taxi-v3')
print(env.P[0][3])
print(env.observation_space.n, env.action_space.n)

[(1.0, 0, -1, False)]
500 6


In [16]:
# Value Iteration
optimal_v_values = value_iteration(env, max_iters=500, gamma=0.9)
optimal_policy = policy_extraction(env, optimal_v_values)
play_multiple_times(env, optimal_policy, 1000)

Converged at 116-th iteration.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  This is separate from the ipykernel package so we can avoid doing imports until


Number of successes: 1000/1000
Average number of steps: 13.204


In [17]:
# Policy Iteration
optimal_p_values = policy_iteration(env)[1]
play_multiple_times(env, optimal_p_values, 1000)

Converged at 28-th iteration.
Number of successes: 1000/1000
Average number of steps: 13.129


# Nhận xét: 
- Về số ván thắng và số bước để chiến thắng của 2 thuật toán Value Iteration và Policy Iteration trên cả 3 map là tương đương nhau
- Về số lần lặp thì Policy Iteration cho tốc độ hội tụ nhanh hơn so với Value Iteration